In [2]:
import random
from lang import load_data_int_seq
from utils import eq_encoder
from models.rl.env import encodeWithLanguage, decodeWithLanguage, createInitialState
%load_ext autoreload
%autoreload 2

In [3]:
class Node:
    def __init__(self, key):
        self.left = None
        self.right = None
        self.val = key

    def __str__(self):
        return "Value: " + str(self.val) + ", Left: " + str(self.left) + ", Right: " + str(self.right)


def insert(root, key):
    if root is None:
        return Node(key)
    elif root.left is None:
        root.left = insert(root.left, key)
    elif root.right is None:
        root.right = insert(root.right, key)
    return root



def generate_possibilities(char: str):
    def list_possibilities(prev: str) -> str:
        if prev in ['1','2','3','4','5','6','7','8','9','0']:
            return ['+', '-', '*']
        elif prev in ['s', '+', '-', '*']:
            return ['1','2','3','4','5','6','7','8','9','0', 't']
        elif prev in ['t']:
            return ['+', '-', '*']
        else:
            raise ValueError('Unexpected prev character')
    lhs = list_possibilities(char)
    rhs = list_possibilities(char)
    return lhs, rhs

In [4]:
generate_possibilities('*')

(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 't'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 't'])

In [5]:
root = Node(random.choice(['*', '+', '-']))
print(root)


Value: +, Left: None, Right: None


In [6]:
root.left = Node(1)
print(root)

Value: +, Left: Value: 1, Left: None, Right: None, Right: None


In [7]:
for possibilities in generate_possibilities(root.val):
    insert(root, random.choice(possibilities))

In [8]:
print(root.right.val)

3


In [6]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [7]:
state = train[1]
eq_encoder('t')

[13]

In [11]:
encodeWithLanguage(input_lang, ['1','2','3','4'])

[57, 43, 32, 41]

In [12]:
decodeWithLanguage(input_lang, [57, 43, 32, 41])

['1', '2', '3', '4']

In [17]:
# 1. [[-1,-1,-1], [1,2,3,4,5,6,7,8,9]]
# action: 1
# 2. [[1,-1,-1], [1,2,3,4,5,6,7,8,9]]
# action: 2
# 2. [[1,2,-1], [1,2,3,4,5,6,7,8,9]]
# action: 3
# 2. [[1,2,3], [1,2,3,4,5,6,7,8,9]]


createInitialState(input_lang, X_test[1], 9)

([-1, -1, -1, -1, -1, -1, -1, -1, -1], [5, 108, 134, 353, 438, 372, 631, 203])